## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


The authors decribe in their paper, which tooles were used for different steps in a RNAseq pipeline, however they miss e.g. versions of these tooles or performed preprocessing steps before alignment.

By using available pipelines of nf-core the whole process will become more reproducible.

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

The next required steps are:
- FastQC to get the quality of the FastQ files
- If trimming is possible, we have to trim the files at the ends of the reads to only have the reads portion with high quality
- Then the reads have to be aligned to a reference genome (not specified in the paper, so we use just the most recent one GRCm39)
- After this aligned reads have to be converted into count numbers for each gene of the annotated reference genome.

In general there is only limited information on versions of tooles used or even the tools are not specified at all. This is why we will use the nf-core/rnaseq pipeline: https://nf-co.re/rnaseq/3.21.0/ .

The pipeline takes a samplesheet with FASTQ files or pre-aligned BAM files as input, performs quality control (QC), trimming and (pseudo-)alignment, and produces a gene expression matrix and extensive QC report.

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?

Which are the main tools that will be used in the pipeline?


1. Merge re-sequenced FastQ files (cat)
2. Auto-infer strandedness by subsampling and pseudoalignment (fq, Salmon)
3. Read QC (FastQC)
4. UMI extraction (UMI-tools)
5. Adapter and quality trimming (Trim Galore!)
6. Removal of genome contaminants (BBSplit)
7. Removal of ribosomal RNA (SortMeRNA)
8. Choice of multiple alignment and quantification routes (For STAR the sentieon implementation can be chosen):
        STAR -> Salmon  
        STAR -> RSEM  
        HiSAT2 -> NO QUANTIFICATION  
    Since HiSAT2 was used in the paper, and we want to reproduce the paper as good as possible, we will also stick to this here.  
9. Sort and index alignments (SAMtools)
10. UMI-based deduplication (UMI-tools)
11. Duplicate read marking (picard MarkDuplicates)
12. Transcript assembly and quantification (StringTie)
13. Create bigWig coverage files (BEDTools, bedGraphToBigWig)
14. Extensive quality control:
        RSeQC
        Qualimap
        dupRadar
        Preseq
        DESeq2
        Kraken2 -> Bracken on unaligned sequences; optional
15. Pseudoalignment and quantification (Salmon or ‘Kallisto’; optional)
16. Present QC for raw read, alignment, gene biotype, sample similarity, and strand-specificity checks (MultiQC, R)


As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [1]:
# prepare a samplesheet with your input data that looks as follows: samplesheet.csv:
# sample,fastq_1,fastq_2,strandedness
# CONTROL_REP1,AEG588A1_S1_L002_R1_001.fastq.gz,AEG588A1_S1_L002_R2_001.fastq.gz,auto
# CONTROL_REP1,AEG588A1_S1_L003_R1_001.fastq.gz,AEG588A1_S1_L003_R2_001.fastq.gz,auto
# CONTROL_REP1,AEG588A1_S1_L004_R1_001.fastq.gz,AEG588A1_S1_L004_R2_001.fastq.gz,auto

# Samplesheet from fetchngs output adaptation to get the required format as described above
import pandas as pd

df = pd.read_csv("/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/samplesheet/samplesheet.csv")

print(df.head())

        sample                                            fastq_1  \
0  SRX19144486  ./SRFetch_results/fastq/SRX19144486_SRR2319551...   
1  SRX19144488  ./SRFetch_results/fastq/SRX19144488_SRR2319551...   

                                             fastq_2 run_accession  \
0  ./SRFetch_results/fastq/SRX19144486_SRR2319551...   SRR23195516   
1  ./SRFetch_results/fastq/SRX19144488_SRR2319551...   SRR23195511   

  experiment_accession sample_accession secondary_sample_accession  \
0          SRX19144486     SAMN32880530                SRS16557078   
1          SRX19144488     SAMN32880528                SRS16557080   

  study_accession secondary_study_accession submission_accession  ...  \
0     PRJNA926667                 SRP418759           SRA1578893  ...   
1     PRJNA926667                 SRP418759           SRA1578893  ...   

  scientific_name sample_title  \
0    Mus musculus           H2   
1    Mus musculus           C3   

                                    experiment_

In [7]:
# Filter the file for the first 3 columns and add the strandedness column with "auto" value
df_final = df.iloc[:, :3]
df_final["strandedness"] = "auto"

# Since FastQ files were copied from a Peer and not downloaded directly from SRA, the filepaths are different
# Adjust paths manually here:
df_final["fastq_1"] = df["fastq_1"].str.replace("./SRFetch_results/fastq/",
                                                "./SRR_data_fetch/fastq/")
df_final["fastq_2"] = df["fastq_2"].str.replace("./SRFetch_results/fastq/",
                                                "./SRR_data_fetch/fastq/")

# To choose indicative samplenames for the two samples, adjust the samplesheet:
# Patient  RNA-seq  DNA-seq Condition Genotype       Bases
# Run                                                                 
# SRR23195516       ?     True    False       Oxy      SNI  6203117700 -> SRX19144486
# SRR23195511       ?     True    False       Oxy     Sham  6456390900 -> SRX19144488

# Rename the sample column:
df_final.loc[df_final['sample'].str.contains('SRX19144486'), 'sample'] = 'Oxy_SNI'
df_final.loc[df_final['sample'].str.contains('SRX19144488'), 'sample'] = 'Oxy_Sham'

print(df_final.head())

df_final.to_csv("/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/samplesheet_rnaseq.csv", index=False)

print("Samplesheet for nf-core/rnaseq pipeline created.")

     sample                                            fastq_1  \
0   Oxy_SNI  ./SRR_data_fetch/fastq/SRX19144486_SRR23195516...   
1  Oxy_Sham  ./SRR_data_fetch/fastq/SRX19144488_SRR23195511...   

                                             fastq_2 strandedness  
0  ./SRR_data_fetch/fastq/SRX19144486_SRR23195516...         auto  
1  ./SRR_data_fetch/fastq/SRX19144488_SRR23195511...         auto  
Samplesheet for nf-core/rnaseq pipeline created.


In [14]:
# When trying to run nextflow nf-core/rnaseq, the sample file paths could not be found properly:

import pandas as pd

df_final_readin = pd.read_csv("/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/samplesheet_rnaseq.csv")
df_final_readin["fastq_1"] = df_final_readin["fastq_1"].apply(lambda x: f"'{x}'")
df_final_readin["fastq_2"] = df_final_readin["fastq_2"].apply(lambda x: f"'{x}'")

print(df_final_readin.head())

df_final.to_csv("/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/samplesheet_rnaseq.csv", index=False)

print("Samplesheet for nf-core/rnaseq pipeline created.")

     sample                                            fastq_1  \
0   Oxy_SNI  './SRR_data_fetch/fastq/SRX19144486_SRR2319551...   
1  Oxy_Sham  './SRR_data_fetch/fastq/SRX19144488_SRR2319551...   

                                             fastq_2 strandedness  
0  './SRR_data_fetch/fastq/SRX19144486_SRR2319551...         auto  
1  './SRR_data_fetch/fastq/SRX19144488_SRR2319551...         auto  
Samplesheet for nf-core/rnaseq pipeline created.


In [ ]:
# post here the command you used to run nf-core/rnaseq

# Manually search for a suitable GTF and genome FASTA file for the mouse genome GRCm39
# (https://www.gencodegenes.org/mouse/release_M10.html)
!nextflow run nf-core/rnaseq \
    --input '/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/samplesheet_rnaseq.csv' \
    --outdir './rnaseq_output' \
    --gtf '/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/gencode.vM10.chr_patch_hapl_scaff.annotation.gtf.gz' \
    --fasta '/home/chrissi/BioPrak/computational-workflows-2025/notebooks/day_02/SRR_data_fetch/GRCm38.p4.genome.fa.gz' \ 
    -profile docker

# Still the pipeline is not able to fined the FastQ files, even though the paths are correct.

Explain all the parameters you set and why you set them in this way.



--input: to specify the samplesheet  
--gtf: download the annotationfile for the mouse genome (website specified above) and reference it here  
--fasta: download the fasta-file for the mouse genome (website specified above) and reference it here  
'- profile docker: to run it in Docker  

## Browsing the results

How did the pipeline perform?

Since we were not able to run the pipeline, it was not possible to evaluate the performance of the pipeline.

Explain the quality control steps. Are you happy with the quality and why. If not, why not.
Please give additional information on : 
- ribosomal rRNA
- Duplication
- GC content

What are the possible steps that could lead to poorer results?

We together had a look at the FastQC and found that 2 samples failed the Quality control.

Would you exclude any samples? If yes, which and why?

The QC failed samples could be excluded because bad quality will affect the downstream analysis. Hoswever this has to be considered very wisely.

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

We can have a look the differential abundance of genes among different groups and try to correlate these DEGs to for example biological pathways.  
For this we can use DESeq2 and GSEA.